구글 드라이브 마운트(Detectron2 패키지 설치후 런타임 다시 시작하면 다시 마운트)

In [1]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


파이썬 패키지가 포함된 폴더 불러오기

In [2]:
import sys
my_path = '/content/drive/MyDrive/additional_package' # 파이썬 패키지가 포함된 폴더 입력
sys.path.append(my_path)
print(sys.path)

['/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/additional_package']


코드 실행에 필요한 패키지 불러오기

In [3]:
import torch, torchvision
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

torch:  1.12 ; cuda:  cu113


In [4]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import math
from math import pi
from google.colab.patches import cv2_imshow
import glob
import pandas as pd
pd.set_option('mode.chained_assignment',  None)
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.detection_utils import read_image
from detectron2.engine import DefaultTrainer

테스트 이미지의 경로와 실행 결과를 저장하고자 하는 경로 입력

In [5]:
VAR_LAYER_CNT = 101
# WORK_DIR = '/content/drive/MyDrive/ML/JTS/'
WORK_DIR = '/content/drive/MyDrive/ML/forena/'
VAR_RES_DIR = WORK_DIR + 'result' # 실행 결과를 저장할 경로
VAR_OUTPUT_DIR = WORK_DIR + 'output' # 모델이 저장된 경로
VAR_PER_BATCH = 12
# class_name = ['button']
class_name = ['button', 'B5', 'B4', 'B3', 'B2', 'B1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
              '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']
VAR_NUM_CLASSES = len(class_name)
VAR_IMAGE_DIR = WORK_DIR + 'test' # 테스트 이미지 경로
VAR_LABEL_DIR = WORK_DIR + 'label' # 라벨링 결과 저장 경로
VAR_AUTOLABEL_DIR = WORK_DIR + 'autoresult' # 라벨링 결과 이미지 저장 경로
VAR_DATA_DIR = WORK_DIR + 'data' # 객체의 위치 데이터 저장 경로
VAR_SAMPLE_DIR = WORK_DIR + 'sample' # 샘플 데이터의 경로

In [6]:
f_path = VAR_IMAGE_DIR # 테스트 이미지 경로
r_path = VAR_RES_DIR # 실행 결과를 저장할 경로
m_path = VAR_OUTPUT_DIR # 모델이 저장된 경로
l_path = VAR_LABEL_DIR # 라벨링 결과 저장 경로
d_path = VAR_DATA_DIR # 객체의 위치 데이터 저장 경로
s_path = VAR_SAMPLE_DIR # 샘플 데이터의 경로
a_path = VAR_AUTOLABEL_DIR # 라벨링 결과 이미지 저장 경로
fnames = os.listdir(f_path) # 이미지 리스트
metadata = MetadataCatalog.get("mdata_").set(thing_classes=class_name)

인식 세팅 및 모델 가중치 불러오기

In [7]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))

cfg.SOLVER.IMS_PER_BATCH = VAR_PER_BATCH
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = VAR_NUM_CLASSES 

cfg.MODEL.WEIGHTS = os.path.join(m_path, "model_final.pth")

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3  # set a custom testing threshold

cfg.freeze()

predictor = DefaultPredictor(cfg)

[10/01 12:04:17 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia

분석 툴

In [8]:
# 객체 간 곂치는 넓이 계산
def compute_intersect_area(rect1, rect2):
  x1, y1 = rect1[0], rect1[1] 
  x2, y2 = rect1[2], rect1[3]
  x3, y3 = rect2[0], rect2[1] 
  x4, y4 = rect2[2], rect2[3]

  ## case1 오른쪽으로 벗어나 있는 경우
  if x2 < x3:
      return 0

  ## case2 왼쪽으로 벗어나 있는 경우
  if x1 > x4:
      return 0

  ## case3 위쪽으로 벗어나 있는 경우
  if  y2 < y3:
      return 0

  ## case4 아래쪽으로 벗어나 있는 경우
  if  y1 > y4:
      return 0

  left_up_x = max(x1, x3)
  left_up_y = max(y1, y3)
  right_down_x = min(x2, x4)
  right_down_y = min(y2, y4)

  width = right_down_x - left_up_x
  height =  right_down_y - left_up_y

  return width * height

In [9]:
# 클러스터링(DBSCAN)
def clustering(df):
  # 정규화
  scaler = StandardScaler()
  sample_scale = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)

  # clustering DBSCAN method
  model = DBSCAN(eps=0.5, min_samples=2)
  model.fit(sample_scale)
  return model.fit_predict(sample_scale)

In [10]:
# PCA
def direction(df):
  X = np.array(df)
  pca = PCA(n_components=2)
  pca.fit(X)

  PCAtheta = np.arctan2(pca.components_[0][0], pca.components_[0][1])/pi # 열의 방향
  PCAlenth = np.sqrt(pca.explained_variance_[0]) # 열의 길이

  return pca.mean_, PCAtheta, PCAlenth

In [11]:
def buttonmatch():
  df1 = df_ins.drop(df_ins[df_ins['class'] != 'button'].index)
  df1 = df1.reset_index(drop=True)
  df2 = df_ins.drop(df_ins[df_ins['class'] == 'button'].index)
  df2 = df2.reset_index(drop=True)

  for i in range(len(df1)):
    box1 = df1.loc[i, 'x1':'y2']
    box1 = np.array(box1, dtype=float)
    area1 = compute_intersect_area(box1, box1)
    for j in range(len(df2)):
      box2 = df2.loc[j, 'x1':'y2']
      box2 = np.array(box2, dtype=float)
      area2 = compute_intersect_area(box1, box2)
      if area2/area1 > 0.8:
        df1.loc[i, 'class'] = df2.loc[j, 'class']
  
  return df1

In [12]:
# sorting
def sortbutton(df, dir, ascd):
  if abs(dir) <= 0.25 or abs(dir >= 0.75): sortvalue = 'Y'
  else: sortvalue = 'X'

  df = df.sort_values(by=sortvalue, ascending=ascd)
  return df.reset_index(drop=True)

def sortline(df):
  df = df.sort_values(by='num_class', ascending=False)
  num_lines = np.unique(df['num_class'])
  for num in num_lines:
    lines = df.loc[df['num_class'] == num]
    if len(lines) == 1: continue
    line_dir = np.mean(abs(lines['theta']))
    if line_dir <= 0.25 or line_dir >= 0.75: sortvalue = 'meanX'
    else: sortvalue = 'meanY'
    df.loc[df['num_class'] == num] = lines.sort_values(by=sortvalue, ascending=True)

  return df.reset_index(drop=True)

In [13]:
class AutoLabel:
  global df_sample
  global df_layout
  global df_ins
  global df_lines
  
  df_sample = pd.DataFrame()
  df_layout = pd.DataFrame()
  df_ins = pd.DataFrame()
  df_lines = pd.DataFrame()

  def __init__(self): # 샘플로부터 버튼의 레이아웃 추출
    global df_sample
    global df_layout
    global df_ins
    global df_lines

    if settings[2] == 'Y':
      # load sample
      with open(s_path + '/sample.json') as f:
        sample = json.load(f) 

      print('Num of sample class: ', len(sample['shapes']))

      for i in range(len(sample['shapes'])):
        poly = np.array(sample['shapes'][i]['points'])
        x, y = poly.mean(axis=0)
        df_sample.loc[i, 'class'] = sample['shapes'][i]['label']
        df_sample.loc[i, 'X'] = x
        df_sample.loc[i, 'Y'] = y

      # 버튼의 열 찾기
      df_sample['cluster'] = clustering(df_sample[['X', 'Y']])

      # 열의 방향 찾기
      for i in range(-1, df_sample['cluster'].max() + 1):
        df1 = df_sample.loc[df_sample['cluster'] == i]
        if len(df1) == 0: continue
        xy, v, l = direction(df1[['X', 'Y']])

        # 열의 버튼을 순서대로 나열
        ascd = True
        df1 = sortbutton(df1, v, ascd)
        df_sample.loc[df_sample['cluster'] == i] = df1

        # 각 열의 정보 저장
        df_layout.loc[i, 'meanX'] = xy[0]
        df_layout.loc[i, 'meanY'] = xy[1]
        df_layout.loc[i, 'theta'] = v
        df_layout.loc[i, 'length'] = l
        # df_layout.loc[i, 'class'] = [df1['class']]
        df_layout.loc[i, 'num_class'] = len(df1['class'])
        df_layout.loc[i, 'ascending'] = ascd

      # 열을 요소의 수와 위치를 기반으로 정렬
      df_layout = sortline(df_layout)

  def run_on_image(self, img): # 이미지에 예측 실행
    global df_sample
    global df_layout
    global df_ins
    global df_lines

    num_instances = 0
    vis_output = None
    predictions = predictor(img)
    image = img[:, :, ::-1]
    v = Visualizer(image, metadata, scale=1.2)
    if "instances" in predictions:
      # 인식 결과
      instances = predictions["instances"].to("cpu")
      v_output = v.draw_instance_predictions(predictions=predictions["instances"].to("cpu"))

      # 인식 결과를 데이터 프레임으로 저장
      class_num = instances.pred_classes.numpy()
      boxes = instances.pred_boxes.tensor.numpy()
      num_instances = len(instances)

      # 객체에 key 값 부여, 클래스 번호를 이름으로 변환
      # x,y pos 및 박스 정보 저장
      for i in range(num_instances):
        df_ins.loc[i, 'key'] = i
        df_ins.loc[i, 'class'] = class_name[class_num[i]]
        df_ins.loc[i, 'X'] = (boxes[i][0] + boxes[i][2]) / 2
        df_ins.loc[i, 'Y'] = (boxes[i][1] + boxes[i][3]) / 2
        df_ins.loc[i, ['x1', 'y1', 'x2', 'y2']] = boxes[i]

    return num_instances, v_output

  def preprocessing(self):
    global df_sample
    global df_layout
    global df_ins
    global df_lines

    crit = df_ins.drop(df_ins[df_ins['class'] == 'button'].index)
    if len(crit):
      df_ins = buttonmatch()
      # df_ins = df_ins.reset_index(drop=True)

    # clustering
    df_ins['cluster'] = clustering(df_ins[['X', 'Y']])

    # PCA
    for i in range(df_ins['cluster'].max() + 1):
      df1 = df_ins.loc[df_ins['cluster'] == i]
      if len(df1) <= 1: continue # 아웃라이어 제거

      xy, v, l = direction(df1[['X', 'Y']])

      # ascd = df_layout.loc[df_layout['num_class'] == len(df1), ['ascending']].drop_duplicates('ascending').to_numpy()
      ascd = True
      df1 = sortbutton(df1, v, ascd)
      df_ins.loc[df_ins['cluster'] == i] = df1

      df_lines[['meanX', 'meanY', 'theta', 'length', 'cluster', 'num_class']] = [xy[0], xy[1], v, l, i, len(df1)]

      # 업데이트 필요(유사도에 따라서 열 매칭)
      # sim = []
      # for x2, y2 in zip(df_layout['meanX'], df_layout['meanY']):
      #   Dx = x2 - xy[0]
      #   Dy = y2 - xy[1]
      #   dist = np.linalg.norm([Dx, Dy])
      #   sim.append(dist)
      # 
      # df_lines['similarity'] = [sim]

    df_lines = sortline(df_lines)

  def assignment(self): # 버튼 자체만 인식된 경우 sample을 기반으로 층 이름 자동 할당
    global df_sample
    global df_layout
    global df_ins
    global df_lines

    cnt = 0
    for cluster in df_lines['cluster']:
      df1 = df_ins.loc[df_ins['cluster'] == cluster]
      df1['class'] = df_sample.loc[df_sample['cluster'] == cluster]['class']
      df_ins.loc[df_ins['cluster'] == cluster] = df1
      cnt += 1

  def inspect(self): # 버튼과 층이 같이 인식된 경우 둘을 조합하여 인식 에러 검사 및 정정
    global df_sample
    global df_layout
    global df_ins
    global df_lines

    for i in range(int(df_ins['cluster'].max()) + 1):
      df1 = df_ins.loc[df_ins['cluster'] == i]
      if len(df1.drop(df1.loc[df1['class'] == 'button'].index)) > 1:
        num_inter = []
        inter_cluster = []
        for j in range(int(df_sample['cluster'].max()) + 1):
          set1 = df1['class']
          set2 = df_sample.loc[df_sample['cluster'] == j, 'class']
          num_inter.append(len(np.intersect1d(set1, set2)))
          inter_cluster.append(j)
        intermax = inter_cluster[np.argmax(num_inter)]

        len1 = len(df1)
        len2 = len(df_sample.loc[df_sample['cluster'] == intermax])
        len2 = int(len2)
        compare = []
        for j in range(len1 + len2 - 1):
          a1 = max([len1 - j - 1, 0])
          b1 = min([len1 + len2 - j -1, len1])
          a2 = max([j - len1 + 1, 0])
          b2 = min([j + 1, len2])
          set1 = df1['class'][a1:b1]
          set1 = set1.reset_index(drop=True)
          set2 = df_sample.loc[df_sample['cluster'] == intermax, 'class'][a2:b2]
          set2 = set2.reset_index(drop=True)
          inter_num = len(set1.compare(set2))
          if a2 > 0:
            inter_num += a2
          if b2 < len2:
            inter_num += len2 - b2
          compare.append(inter_num)

        print('compare', compare)
        idx = np.argmin(compare)

        a1 = max([len1 - idx - 1, 0])
        b1 = min([len1 + len2 - idx -1, len1])
        a2 = max([idx - len1 + 1, 0])
        b2 = min([idx + 1, len2])
        print('A', df1['class'][a1:b1])
        print('B', df_sample.loc[df_sample['cluster'] == i, 'class'][a2:b2])
        df1['class'][a1:b1] = df_sample.loc[df_sample['cluster'] == i, 'class'][a2:b2]

        df_ins.loc[df_ins['cluster'] == i] = df1

  def auto_label(self, img, fname, f_name): # 인식 결과를 json 파일로 저장
    h, w, c = img.shape
    label_data = {
        "version": "4.6.0",
        "flags":{},
        "shapes": [],
        "imagePath": fname,
        "imageData": None,
        "imageHeight": h,
        "imageWidth": w
        }
    for i in range(len(df_ins)):
      class_name = df_ins.loc[i, 'class']
      box = df_ins.loc[i, 'x1':'y2']
      label_data["shapes"].append({
          "label": class_name,
          "points": [
              [float(box[0]),
               float(box[1])
              ],
              [float(box[2]),
               float(box[3])   
              ],
          ],
          "group_id": None,
          "shape_type": "rectangle", # polygon 형태로 저장
          "flags": {}
      }
      )

    os.makedirs(l_path, exist_ok=True)
    with open(l_path + '/' + f_name + '.json', 'w') as fp:
      json.dump(label_data, fp, indent=2)
    print(f_name, len(df_ins), 'labeled')

  def data_save(self, count): # 인식 결과를 excel로 저장(이미 같은 이름의 excel 파일이 있는 경우 에러 발생)
    os.makedirs(d_path, exist_ok=True)
    if not os.path.exists(d_path + '/pos_data.xlsx'):
      with pd.ExcelWriter(d_path + '/pos_data.xlsx', mode='w', engine='openpyxl') as writer:
        df_ins.to_excel(writer, index=False, sheet_name=str(count-1))

    else:
      with pd.ExcelWriter(d_path + '/pos_data.xlsx', mode='a', engine='openpyxl') as writer:
        df_ins.to_excel(writer, index=False, sheet_name=str(count-1))

테스트 이미지 불러오기 및 결과 저장

In [14]:
def f_data_test(settings):
  Auto = AutoLabel()
  total_f = len(glob.glob(f_path + '/*.jpg'))
  count = 1
  
  # 모든 테스트 이미지에 예측 실행
  for fname in fnames:
    f_name, f_ext = os.path.splitext(fname)
    filename = os.path.join(f_path, fname)
    print('filename: ', filename)

    # jpg or JPEG가 아닌 파일이 있는 경우 스킵
    if (f_ext != '.jpg' and f_ext != '.JPEG'): continue

    img = read_image(filename, format='BGR')
    num_instances, v_output= Auto.run_on_image(img)
    print('progress: ', count, '/', total_f)
    print(num_instances, 'detected')
    count += 1

    if num_instances == 0: continue

    # Auto labeling 진행
    if settings[0] == '2':
      # Auto labeling 전에 에러 검사 및 정정
      if settings[2] == 'Y':
        Auto.preprocessing()
        # print('sample', df_sample)
        # print('ins', df_ins)
        crit = df_ins.drop(df_ins[df_ins['class'] == 'button'].index)
        if len(crit): Auto.inspect()
        else: Auto.assignment()

      Auto.auto_label(img, fname, f_name)

    # 수정 전 인식 결과 이미지 저장
    if settings[1] == 'Y':
      if num_instances > 0: # save in files
        if os.path.isdir(r_path):
          assert os.path.isdir(r_path), r_path
        else:
          os.makedirs(r_path, exist_ok=True)

        out_filename = os.path.join(r_path, fname)
        v_output.save(out_filename)
    
    # 인식 결과를 엑셀로 저장
    if settings[3] == 'Y':
      Auto.data_save(count)
  
  print('testing done')

실행 옵션

In [15]:
def set_option():
  print('help - data test(1), auto labeling(2)')
  opt_1 = input('Option: ') # 1: test only, 2: auto label
  opt_2 = input('Save Result(Y/N): ') # 인식 결과 저장 여부
  if opt_1 == '2':
    opt_3 = input('Assist Program On(Y/N): ') # 어시스트 프로그램 ON/OFF
  else:
    opt_3 = 'N'
  opt_4 = input('Save Data(Y/N): ') # 인식 결과 데이터 저장 여부
  return opt_1, opt_2, opt_3, opt_4

메인 함수 실행

In [16]:
if __name__ == '__main__':
  settings = set_option()
  f_data_test(settings)

help - data test(1), auto labeling(2)
Option: 2
Save Result(Y/N): Y
Assist Program On(Y/N): N
Save Data(Y/N): N
filename:  /content/drive/MyDrive/ML/forena/test/20220826_183431_015.jpg


/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


progress:  1 / 300
16 detected
20220826_183431_015 16 labeled
filename:  /content/drive/MyDrive/ML/forena/test/20220826_183431_014.jpg
progress:  2 / 300
20 detected
20220826_183431_014 20 labeled
filename:  /content/drive/MyDrive/ML/forena/test/20220826_183431_013.jpg
progress:  3 / 300
19 detected
20220826_183431_013 20 labeled
filename:  /content/drive/MyDrive/ML/forena/test/20220826_183431_012.jpg
progress:  4 / 300
17 detected
20220826_183431_012 20 labeled
filename:  /content/drive/MyDrive/ML/forena/test/20220826_183431_011.jpg
progress:  5 / 300
15 detected
20220826_183431_011 20 labeled
filename:  /content/drive/MyDrive/ML/forena/test/20220826_183431_005.jpg
progress:  6 / 300
19 detected
20220826_183431_005 20 labeled
filename:  /content/drive/MyDrive/ML/forena/test/20220826_183431_004.jpg
progress:  7 / 300
19 detected
20220826_183431_004 20 labeled
filename:  /content/drive/MyDrive/ML/forena/test/20220826_183431_010.jpg
progress:  8 / 300
17 detected
20220826_183431_010 20 l

labeled data(json 파일) 읽기

In [17]:
# labelme에서 만든 이미지 labeling 에서 정보를 가져와서 추출하는 코드
def dataset_func(dir):
  import os, json
  import numpy as np
  from detectron2.structures import BoxMode
  
  dataset_dict = []
  afile = os.listdir(dir)
  for jsonf in afile:
      print(jsonf)
      t = jsonf.find('.json')
      if (t>=0):
        print('filename:', jsonf)
      else: continue
      
      jsonf=dir+'/'+jsonf
      print(jsonf)
      with open(jsonf, encoding="utf8", errors='ignore') as f:
        imgs_anns = json.load(f)
        print(imgs_anns)
        
      fname = os.path.join(dir, imgs_anns['imagePath']) 
      print(fname)
      
      record={}
      record['file_name']=fname
      record['image_id']=0
      record['height']=imgs_anns['imageHeight']
      record['width']=imgs_anns['imageWidth']
      
      objs=[]
      for v in imgs_anns['shapes']:
        poly = v['points']
        if v['shape_type'] == 'rectangle':
          xmin = poly[0][0]
          xmax = poly[1][0]
          ymin = poly[0][1]
          ymax = poly[1][1]
          poly = np.insert(poly, 1, [xmin, ymax], axis=0)
          poly = np.insert(poly, 3, [xmax, ymin], axis=0)
        poly = [p for x in poly for p in x]

######## 클래스 추가 #########
        if(v['label']=='button'): cid = 0
        elif(v['label']=='B5'): cid = 1
        elif(v['label']=='B4'): cid = 2
        elif(v['label']=='B3'): cid = 3
        elif(v['label']=='B2'): cid = 4
        elif(v['label']=='B1'): cid = 5
        elif(v['label']=='1'): cid = 6
        elif(v['label']=='2'): cid = 7
        elif(v['label']=='3'): cid = 8
        elif(v['label']=='4'): cid = 9
        elif(v['label']=='5'): cid = 10
        elif(v['label']=='6'): cid = 11
        elif(v['label']=='7'): cid = 12
        elif(v['label']=='8'): cid = 13
        elif(v['label']=='9'): cid = 14
        elif(v['label']=='10'): cid = 15
        elif(v['label']=='11'): cid = 16
        elif(v['label']=='12'): cid = 17
        elif(v['label']=='13'): cid = 18
        elif(v['label']=='14'): cid = 19
        elif(v['label']=='15'): cid = 20
        elif(v['label']=='16'): cid = 21
        elif(v['label']=='18'): cid = 22
        elif(v['label']=='19'): cid = 23
        elif(v['label']=='20'): cid = 24
        elif(v['label']=='21'): cid = 25
        elif(v['label']=='22'): cid = 26
        elif(v['label']=='23'): cid = 27
        elif(v['label']=='24'): cid = 28
        elif(v['label']=='25'): cid = 29
        elif(v['label']=='26'): cid = 30
        elif(v['label']=='27'): cid = 31
        elif(v['label']=='28'): cid = 32
        elif(v['label']=='29'): cid = 33
        else: cid = -1 #No category
    
        x1,y1 = np.min(poly[0::2]),np.min(poly[1::2])
        xe,ye = np.max(poly[0::2]),np.max(poly[1::2])
        obj = {
          'bbox': [x1,y1,xe,ye],
          'bbox_mode': BoxMode.XYXY_ABS,
          'segmentation': [poly],
          'category_id': cid
        }
        if cid == -1:
          continue
        objs.append(obj)

      record['annotations']=objs
      dataset_dict.append(record)

  print(dataset_dict)
  return(dataset_dict)

본인의 데이터 Register

In [18]:
class_name = ['button', 'B5', 'B4', 'B3', 'B2', 'B1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
              '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']

# class_name = ['button']

for d in ["train", "val"]:
    DatasetCatalog.register("mdata_" + d, lambda d=d: dataset_func(WORK_DIR + d))
    MetadataCatalog.get("mdata_" + d).set(thing_classes=class_name)
my_metadata = MetadataCatalog.get("mdata_train")

In [ ]:
# Dataset 삭제
DatasetCatalog.clear()
MetadataCatalog.clear()

본인의 데이터 라벨링 결과 확인

In [19]:
import random

dataset_dicts = dataset_func(WORK_DIR + "train")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.

학습 시작

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
#### 데이터셋 이름 확인 
cfg.DATASETS.TRAIN = ("mdata_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 3000   # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
# 본인의 데이터 분류 클래스 개수 입력
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_name) # only has one class (mount).

cfg.OUTPUT_DIR = WORK_DIR + "output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir WORK_DIR + "output"

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

# # 테스트하기 (섬이 있는 학습되지 않은 이미지에서 결과 확인)
# # 학습한 사물과 유사한 것이 있으면 인식됨 (학습결과가 좋지 않으면 위의 학습 과정에서 cfg.SOLVER.MAX_ITER = 300 을 300 이상으로 올리면 효과가 더 좋아짐)
# im = cv2.imread('/content/drive/MyDrive/ML/elebutton/train/snapshot_640_480_0.jpg')  
# outputs = predictor(im)
# v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
# out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
import cv2

# 테스트하기 (섬이 있는 학습되지 않은 이미지에서 결과 확인)
# 학습한 사물과 유사한 것이 있으면 인식됨 (학습결과가 좋지 않으면 위의 학습 과정에서 cfg.SOLVER.MAX_ITER = 300 을 300 이상으로 올리면 효과가 더 좋아짐)
im = cv2.imread('/content/drive/MyDrive/ML/forena/test/20220826_183431_001.jpg')
outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.